<a href="https://colab.research.google.com/github/phoekoby/StegoanalysisNN/blob/master/CNN_XuNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install torch torchvision
# !pip3 install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from collections import namedtuple

import matplotlib.pyplot as plt
import numpy as np
import PIL
import torch
import os
import torch.nn as nn
import torch.optim as optim

import pandas as pd

import torchvision.datasets as dset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.io import read_image


from torchvision import transforms

In [1]:
device = torch.device("cuda:0")


class CustomDataset:
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file, delimiter=' ',header=None)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0][2:])
        image = PIL.Image.open(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return (image, label)

data_train = CustomDataset('/content/drive/MyDrive/dataset/train_annotation.csv', 
                           '/content/drive/MyDrive/dataset/', transform=transforms.ToTensor())
data_test = CustomDataset('/content/drive/MyDrive/dataset/test_annotation.csv',
                          '/content/drive/MyDrive/dataset/', transform=transforms.ToTensor())

batch_size = 128

data_size = 1000
validation_split = .2
split = int(np.floor(validation_split * data_size))
indices = list(range(data_size))
np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(data_train, batch_size=batch_size,
                                         sampler=val_sampler)
class Flattener(nn.Module):
    def forward(self, x):
        batch_size, *_ = x.shape
        return x.view(batch_size, -1)
nn_model = nn.Sequential(
        nn.Conv2d(
            3,
            8,
            5,
            stride=1,
            padding=2,
            bias=False,
        ),
        nn.BatchNorm2d(8),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
        nn.Conv2d(
            8,
            16,
            5,
            stride=1,
            padding=2,
            bias=False,
        ),
        nn.BatchNorm2d(16),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
        nn.Conv2d(
            16,
            32,
            1,
            stride=1,
            padding=0,
            bias=False,
        ),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
        nn.Conv2d(
            32,
            64,
            1,
            stride=1,
            padding=0,
            bias=False,
        ),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
        nn.Conv2d(
            64,
            128,
            1,
            stride=1,
            padding=0,
            bias=False,
        ),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
        nn.AdaptiveAvgPool2d(output_size=1),
        Flattener(),
        nn.Linear(in_features=128, out_features=128),
        nn.ReLU(inplace=True),
        nn.Linear(in_features=128, out_features=2),
        nn.LogSoftmax(dim=1)
  )

nn_model.type(torch.cuda.FloatTensor)
nn_model.to(device)

# loss = nn.CrossEntropyLoss().type(torch.cuda.FloatTensor)
loss = nn.NLLLoss().type(torch.cuda.FloatTensor)
optimizer = optim.Adam(nn_model.parameters(), lr=0.01, betas=(0.99, 0.999), weight_decay=1e-4)
# optimizer = optim.SGD(nn_model.parameters(), lr=1e-1, weight_decay=1e-4)

def train_model(model, train_loader, val_loader, loss, optimizer, num_epochs):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y) in enumerate(train_loader):
          
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value

        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy = compute_accuracy(model, val_loader)
        
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)
        
        print("Average loss: %f, Train accuracy: %f, Val accuracy: %f" % (ave_loss, train_accuracy, val_accuracy))
        
    return loss_history, train_history, val_history
        
def compute_accuracy(model, loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for i_step, (x, y) in enumerate(loader):
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            predictions = model(x_gpu)
            _, predicted = torch.max(predictions, 1)
            total += y_gpu.size(0)
            correct += (predicted == y_gpu).sum().item()
    return correct/total

loss_history, train_history, val_history = train_model(nn_model, train_loader, val_loader, loss, optimizer, 50)

NameError: ignored